In [1]:
import json
from elasticsearch import Elasticsearch

In [2]:
import math
import numpy as np
import datetime

In [3]:
import pandas as pd

In [4]:
import time

In [5]:
data = json.load(open("data_formular/formular.txt"))

In [6]:
type(data)

list

In [7]:
data[:5]

['\\sqrt { 2 x + 7 }',
 '\\sqrt { \\frac { 1 } { - 1 + x } }',
 '\\sqrt { - 3 x + 4 }',
 '\\sqrt { 1 + x ^ { 2 } }',
 '\\sqrt { 0,09.64 }']

In [8]:
len(data)

2623

In [9]:
data_dict = dict()
j = 1
for i in data[:500]:
    print('len i :',len(i.split()))
    i_dict = dict()
    if len(i.split()) > 2:
        i_dict['formular'] = i
        data_dict['e' + str(j)] = i_dict
        print(j)
        j = j+ 1


len i : 7
1
len i : 13
2
len i : 8
3
len i : 10
4
len i : 4
5
len i : 4
6
len i : 13
7
len i : 9
8
len i : 3
9
len i : 9
10
len i : 3
11
len i : 9
12
len i : 3
13
len i : 2
len i : 10
14
len i : 10
15
len i : 10
16
len i : 10
17
len i : 16
18
len i : 4
19
len i : 4
20
len i : 6
21
len i : 4
22
len i : 4
23
len i : 4
24
len i : 4
25
len i : 22
26
len i : 27
27
len i : 3
28
len i : 3
29
len i : 33
30
len i : 72
31
len i : 32
32
len i : 4
33
len i : 1
len i : 1
len i : 1
len i : 1
len i : 1
len i : 4
34
len i : 3
35
len i : 3
36
len i : 21
37
len i : 4
38
len i : 25
39
len i : 3
40
len i : 17
41
len i : 6
42
len i : 17
43
len i : 7
44
len i : 33
45
len i : 12
46
len i : 1
len i : 1
len i : 8
47
len i : 1
len i : 6
48
len i : 3
49
len i : 6
50
len i : 1
len i : 1
len i : 2
len i : 3
51
len i : 1
len i : 1
len i : 3
52
len i : 5
53
len i : 6
54
len i : 5
55
len i : 6
56
len i : 1
len i : 2
len i : 1
len i : 1
len i : 5
57
len i : 3
58
len i : 6
59
len i : 5
60
len i : 6
61
len i : 5
62
len 

In [10]:
len(data_dict)

324

In [11]:
data_dict['e324']

{'formular': '\\sqrt { 25 a ^ { 2 } } + 3 a'}

In [12]:
data[499]

'\\sqrt { 25 a ^ { 2 } } + 3 a'

In [13]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [14]:
# insert 
for i,j in zip(data_dict, range(1, 501)):
    res = es.index(index = 'idform', doc_type = 'form', id = int(j), body = data_dict[i])
    print(res['result'])

created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created


# find the parameters for similarity function
+ make : 
    + take the : search_string
    + count_char_in_the_doc: contain the number of character appearing in the documents that contain term (the characters in search string)
    + length_of_all_document: contain the length of the all documents in ES
    + docFreq : the number of docs contain the term (search_tring)
    + numDocs : the sum of the documents in ES
    + length_avg : the average of the length in ES
    + k = 1.25, b = 0.75 (default)

In [17]:
class search_bm25_02(object):
    def __init__(self, k , b, query_string = None, data_dict = None, es = Elasticsearch([{'host': 'localhost', 'port': 9200}])):
        self.k = k
        self.b = b
        # query string
        self.query_string = query_string
        # original data (format type : dictionary)
        self.data_dict = data_dict
        self.es = es
        
    def setup(self):
        # convert to set take the unique elements, then convert query string to query list (format type : list)
        self.query_list = list(set(self.query_string.split()))
        
        # create a new dictionary, length_of_all_docs, contain the length of all documents in ES
        self.length_of_all_docs = dict()
        
        # docs_formular
        self.docs_formular = dict()
        # create a new dictionary, count_char_in_the_docs, contain the documents those contain the character was appeared in query_string
        self.count_char_in_the_docs = dict()
        
        for i,j in zip(data_dict, range(1, 501)):
            # count a new dict, count_char, contain infor of a document that contain the characters was appeared in query_string
            # and the appearing number of character in this document
            count_char = dict()
            # create a new variable, f_data,get data from ES
            f_data = self.es.get(index = 'idform', doc_type = 'form', id = int(j))
            
            self.length_of_all_docs[str(j)] = len(f_data['_source']['formular'].split())
            self.docs_formular[str(j)] = f_data['_source']['formular']
            
            for f_char in self.query_list:
                a = 0
                # if the character of query string appear in current document
                if f_char in f_data['_source']['formular'].split():
                    a = f_data['_source']['formular'].split().count(f_char)
                    if a> 0:
                        count_char[f_char] = a
            if count_char:
                self.count_char_in_the_docs[str(j)] = count_char
        
        return self.length_of_all_docs, self.count_char_in_the_docs, self.docs_formular
        
        
        
    def IDF(self):
        self.length_of_all_docs, self.count_char_in_the_docs, self.docs_formular = self.setup()
        # create a new variable, numDocs, is the number of all documents in ES
        self.numDocs = len(self.data_dict)
        
        # create a new variable, docFreq, is the documents number contain characters was appeared in query string
        self.docFreq = len(self.count_char_in_the_docs)
        

        num = (self.numDocs - self.docFreq + 0.5) / (self.docFreq + 0.5)
        return math.log(1 + num)
    
    def TF(self, i):
        self.length_of_all_docs, self.count_char_in_the_docs, self.docs_formular = self.setup()
        
        # create a new variabl, length_avg, is the average length of all document lengths
        self.length_avg = sum(self.length_of_all_docs.values())/ len(self.data_dict)
        
        length_of_doc = self.length_of_all_docs[str(i)]
        L = length_of_doc / self.length_avg

        freq = (sum(self.count_char_in_the_docs[str(i)].values()))
        
        return (freq*(self.k + 1)) / (self.k*(1 - self.b + self.b*L) + freq)
    
    
    def score(self, i):
        print('pid :', i)
        scor = self.IDF() * self.TF(i)
        recommend = pd.DataFrame()
        recommend[str(i)] = [scor]
        return recommend.T
            

In [18]:
import re
# filter the special characters in the query_string
def filter_query_string(query_str):
    return re.sub("{|}","", query_str)

In [19]:
#filter_query_string("\\frac { 3 } { 2 + a }")

In [20]:
'''t1 = time.time()
query_set = set(filter_query_string("\\frac { 3 } { 2 + a }").split())
query_list = list(query_set)
print(query_list)
t2 = time.time()
print(t2-t1)'''
'''t1 = time.time()
query_list1 = list(set(filter_query_string("\\frac { 3 } { 2 + a }").split()))
print(query_list1)
t2 = time.time()
print(t2-t1)'''
'''a = 'r'
if a in query_list:
    print(1)
else:
    print(0)'''

"a = 'r'\nif a in query_list:\n    print(1)\nelse:\n    print(0)"

In [21]:
bm25 = search_bm25_02(k = 1.25, b = 0.75, query_string= filter_query_string("\\frac { 3 } { 2 + a }"), data_dict = data_dict, es = Elasticsearch([{'host': 'localhost', 'port': 9200}]))

In [22]:
length_of_all_docs,count_char_in_the_docs, docs_formular = bm25.setup()

In [23]:
length_of_all_docs[str(321)]

14

In [24]:
docs_num_contain_term = list(count_char_in_the_docs.keys())
docs_num_contain_term

['1',
 '2',
 '3',
 '4',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '35',
 '36',
 '37',
 '38',
 '39',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '50',
 '53',
 '54',
 '55',
 '56',
 '59',
 '60',
 '61',
 '62',
 '64',
 '68',
 '69',
 '70',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '96',
 '102',
 '107',
 '108',
 '109',
 '110',
 '113',
 '114',
 '115',
 '116',
 '117',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '132',
 '142',
 '163',
 '168',
 '172',
 '174',
 '179',
 '180',
 '181',
 '182',
 '184',
 '186',
 '190',
 '191',
 '192',
 '193',
 '201',
 '202',
 '204',
 '205',
 '206',
 '207',
 '208',
 '209',
 '210',
 '211',
 '214',
 '215',
 '216',
 '219',
 '221',
 '222',
 '223',
 '224',
 '225',
 '226',
 '227',
 '228',
 '229',
 '230',
 '231',
 '232',
 '233',
 '234',
 '235',
 '236',
 '237',
 '238',
 '239',
 '240',
 '241',
 '242',
 '243',
 '245',
 '246',
 '24

In [25]:
recommendation = pd.DataFrame()

In [26]:
t1 = time.time()

# create a new series, ser_score, contain the id and score of the document in count_char_in_the_docs
for i in  docs_num_contain_term:
    df_score = bm25.score(int(i))
    # make computation the score for the all documents that contain the term (query string)
    recommendation = recommendation.append(df_score)
t2 = time.time()
print('recommend time : {0:0.00f}'.format(t2-t1))

pid : 1
pid : 2
pid : 3
pid : 4
pid : 7
pid : 8
pid : 9
pid : 10
pid : 11
pid : 12
pid : 13
pid : 14
pid : 15
pid : 16
pid : 17
pid : 18
pid : 26
pid : 27
pid : 28
pid : 29
pid : 30
pid : 31
pid : 32
pid : 35
pid : 36
pid : 37
pid : 38
pid : 39
pid : 41
pid : 42
pid : 43
pid : 44
pid : 45
pid : 46
pid : 47
pid : 50
pid : 53
pid : 54
pid : 55
pid : 56
pid : 59
pid : 60
pid : 61
pid : 62
pid : 64
pid : 68
pid : 69
pid : 70
pid : 76
pid : 77
pid : 78
pid : 79
pid : 80
pid : 81
pid : 82
pid : 83
pid : 84
pid : 85
pid : 86
pid : 87
pid : 96
pid : 102
pid : 107
pid : 108
pid : 109
pid : 110
pid : 113
pid : 114
pid : 115
pid : 116
pid : 117
pid : 123
pid : 124
pid : 125
pid : 126
pid : 127
pid : 128
pid : 129
pid : 132
pid : 142
pid : 163
pid : 168
pid : 172
pid : 174
pid : 179
pid : 180
pid : 181
pid : 182
pid : 184
pid : 186
pid : 190
pid : 191
pid : 192
pid : 193
pid : 201
pid : 202
pid : 204
pid : 205
pid : 206
pid : 207
pid : 208
pid : 209
pid : 210
pid : 211
pid : 214
pid : 215
pid : 21

In [27]:
recommendation

,0
1,0.837285
2,0.705589
3,0.812024
4,0.765816
7,0.705589
8,0.788244
9,0.779159
10,0.578277
11,0.779159
12,0.788244


In [28]:
recommendation.columns = [ 'score']
recommendation = recommendation.sort_values(by = ['score'], ascending = False)

recommendation

,score
324,0.964461
50,0.960805
87,0.946159
234,0.939533
265,0.938472
114,0.938472
248,0.937584
84,0.929816
322,0.914029
224,0.914029


In [29]:
recommendation = recommendation.reset_index()
recommendation.columns = ['id', 'score']

In [30]:
recommendation

,id,score
0,324,0.964461
1,50,0.960805
2,87,0.946159
3,234,0.939533
4,265,0.938472
5,114,0.938472
6,248,0.937584
7,84,0.929816
8,322,0.914029
9,224,0.914029


In [31]:
# \\frac { 3 } { 2 + a }
for i in range(0,recommendation.shape[0]):
    a = es.get(index = 'idform', doc_type = 'form', id = int(recommendation.loc[i,'id']))
    print('score : {} & formular : {}'.format(recommendation.loc[i,'score'], a['_source']['formular']))

score : 0.964460810003639 & formular : \sqrt { 25 a ^ { 2 } } + 3 a
score : 0.9608054563541608 & formular : y = 2 x + 2
score : 0.9461591906229757 & formular : 2 \sqrt { 3 } a ^ { 2 }
score : 0.9395333545990482 & formular : \frac { 3 + 2 \sqrt { 3 } } { \sqrt { 3 } } - \frac { 2 \sqrt { 3 } + \sqrt { 15 } } { \sqrt { 5 } + 2 }
score : 0.9384717235647309 & formular : 0 x + 3 y = 3
score : 0.9384717235647309 & formular : 3 x + 2 y = 5
score : 0.9375838218781346 & formular : 3 + \sqrt [ 3 ] { 5 x + 3 } = 0
score : 0.9298163584272041 & formular : \frac { 2 a } { \sqrt { 3 } }
score : 0.9140285134581544 & formular : 2 \sqrt { a ^ { 2 } } - 5 a
score : 0.9140285134581544 & formular : \frac { a } { \sqrt { a } + 1 }
score : 0.900763799042794 & formular : \sqrt { - 9 a } - \sqrt { 9 + 12 a + 4 a ^ { 2 } }
score : 0.8999643595513834 & formular : \sqrt { 1 - 10 a + 25 a ^ { 2 } } - 4 a
score : 0.898767857450134 & formular : x + \frac { b } { 2 a } = \pm \ldots
score : 0.8967807416450275 & formul

# delete index

In [ ]:
# delete
for i in range(1, len(data_dict) + 1):
    print(i)
    res = es.delete(index = 'idform', doc_type = 'form', id = int(i))